# Visualization of QM scan energies (from Torsiondrive) as energy countour, compared to MM energies (from Forcebalance) at the same structures.

In [73]:
#### User input

# scan = "S_dihrot"
# txtafter = "/home/winter/onsager/fb/fbS_nolog/input.tmp/scan_dihrot/iter_0028/EnergyCompare.txt"
# txtbefore = "/home/winter/onsager/fb/fbS_nolog/input.tmp/scan_dihrot/iter_0000/EnergyCompare.txt"

# scan = "S"
# txtafter = "/home/winter/onsager/fb/fbS_nolog/input.tmp/scan_dihpyr/iter_0028/EnergyCompare.txt"
# txtbefore = "/home/winter/onsager/fb/fbS_nolog/input.tmp/scan_dihpyr/iter_0000/EnergyCompare.txt"

# scan = "N_dihrot"
# txtafter = "/home/winter/onsager/fb/fbN_nolog/input.tmp/scan_dihrot/iter_0059/EnergyCompare.txt"
# txtbefore = "/home/winter/onsager/fb/fbN_nolog/input.tmp/scan_dihrot/iter_0000/EnergyCompare.txt"

# scan = "N"
# txtafter = "/home/winter/onsager/fb/fbN_nolog/input.tmp/scan_dihpyr/iter_0059/EnergyCompare.txt"
# txtbefore = "/home/winter/onsager/fb/fbN_nolog/input.tmp/scan_dihpyr/iter_0000/EnergyCompare.txt"

scan = "O_dihrot"
txtafter = "/home/winter/onsager/fb/fbO_2scans/input.tmp/scan_dihrot/iter_0029/EnergyCompare.txt"
txtbefore = "/home/winter/onsager/fb/fbO_2scans/input.tmp/scan_dihrot/iter_0000/EnergyCompare.txt"

# scan = "O_dihrot"
# txtafter = "/home/winter/onsager/fb/fbO_allboxbuckpaper/input.tmp/scan_dihrot/iter_0024/EnergyCompare.txt"
# txtbefore = "/home/winter/onsager/fb/fbO_allboxbuckpaper/input.tmp/scan_dihrot/iter_0000/EnergyCompare.txt"

# scan = "O"
# txtafter = "/home/winter/onsager/fb/fbO_2scans/input.tmp/scan_dihpyr/iter_0029/EnergyCompare.txt"
# txtbefore = "/home/winter/onsager/fb/fbO_2scans/input.tmp/scan_dihpyr/iter_0000/EnergyCompare.txt"

# scan = "O"
# txtafter = "/home/winter/onsager/fb/fbO_allboxbuckpaper/input.tmp/scan_dihpyr/iter_0024/EnergyCompare.txt"
# txtbefore = "/home/winter/onsager/fb/fbO_allboxbuckpaper/input.tmp/scan_dihpyr/iter_0000/EnergyCompare.txt"

scanxyz = "/home/winter/onsager/tsqm/td/"+scan+"/scan.xyz"
scanpdb = "/home/winter/onsager/tsqm/td/"+scan+"/scan.pdb"
import numpy as np
import nglview as nv
from IPython.display import display
import plotly.graph_objs as go
import plotly

# Plot QM energies

In [74]:
# Parse dihedrals and energies. From Yudong's script.
with open(scanxyz) as f:
    lines = f.readlines()
global n_atoms 
n_atoms = int(lines[0])
comment_lines = lines[1::n_atoms+2]
grid_data = dict()
for line in comment_lines:
    ls = line.strip().split()
    assert ls[0] == 'Dihedral' and ls[-2] == 'Energy', line
    grid_energy = float(ls[-1])
    grid_coord = []
    for i in range(1, len(ls) - 2):
        c = int(ls[i].replace('(', '').replace(',','').replace(')',''))
        grid_coord.append(c)
    grid_data[tuple(grid_coord)] = grid_energy

# auto determination of No. of scan points (xpts, ypts), minimum values (xmin, ymin) and step size (xstep,ystep)
i = 0
while lines[1+i*(n_atoms+2)].strip().split()[1] == lines[1+(i+1)*(n_atoms+2)].strip().split()[1] != 0:
    i=i+1
global xpts, ypts, xmin, ymin, xstep, ystep
ypts = i+1
xpts = len(grid_data) // ypts
xmin = float(lines[1].strip().split()[1][1:-1])
ymin = float(lines[1].strip().split()[2][0:-1])
xstep = float(lines[1+ypts*(n_atoms+2)].strip().split()[1][1:-1]) - float(lines[1].strip().split()[1][1:-1])
ystep = float(lines[1+(n_atoms+2)].strip().split()[2][0:-1]) - float(lines[1].strip().split()[2][0:-1])

x = np.linspace(xmin,xmin+xstep*(xpts-1),num=xpts)
y = np.linspace(ymin,ymin+ystep*(ypts-1),num=ypts)
rawe = np.reshape(np.fromiter(grid_data.values(), dtype=float),(xpts,ypts)).T
# convering rawe in hartree to e in kcal/mol with lowest value offset to zero.
e = rawe*627.509-np.min(rawe*627.509)

def myplot(energy):
    fig = go.FigureWidget([go.Contour(x=x, y=y, z=energy, 
                            colorscale='YlGnBu', contours=dict(start=0,end=18,size=1.5),
                            colorbar=dict(nticks=8,title='Energy (kcal/mol)')
                           )],
                      go.Layout(xaxis=dict(title='central torsion'),yaxis=dict(title='rotor pucker'),width=500,height=300,
                               paper_bgcolor='rgba(0,0,0,0)',plot_bgcolor='rgba(0,0,0,0)'))
    return fig

fig = myplot(e) ; fig

FigureWidget({
    'data': [{'colorbar': {'nticks': 8, 'title': {'text': 'Energy (kcal/mol)'}},
              …

In [75]:
rawmme = np.genfromtxt(txtafter, skip_header=1, skip_footer=0, usecols=1)
mme = 0.239*np.reshape(rawmme-np.min(rawmme),(xpts,ypts)).T   # zero offset and convert kj/mol to kcal/mol energies

fig = myplot(mme) ; fig

FigureWidget({
    'data': [{'colorbar': {'nticks': 8, 'title': {'text': 'Energy (kcal/mol)'}},
              …

In [76]:
rawmme = np.genfromtxt(txtbefore, skip_header=1, skip_footer=0, usecols=1)
mme = 0.239*np.reshape(rawmme-np.min(rawmme),(xpts,ypts)).T   # zero offset and convert kj/mol to kcal/mol energies

fig = myplot(mme) ; fig

FigureWidget({
    'data': [{'colorbar': {'nticks': 8, 'title': {'text': 'Energy (kcal/mol)'}},
              …